In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import skew

# Reproducible randomness for bootstrap
rng = np.random.default_rng(42)



### Q1 and Q2

Do a regression to estimate the fixed effect of each group. 

We assume that there is one single linear coefficient for all the data, plus the fixed effect of each group. Use the file homework_2.1.csv.  

The variables G1, G2, and G3 are the outcomes and the time is the treatment.

Question 1
Which of these is closest to being the coefficient of group 1? 

Question 2
Which of these is closest to being the common linear coefficient for all groups?

In [2]:
# --- 📂 Load data ---
p1_path = "homework_2.1.csv"
p2_path = "homework_2.2.csv"

df1 = pd.read_csv(p1_path)
df2 = pd.read_csv(p2_path)

print("homework_2.1.csv shape:", df1.shape)
print("homework_2.2.csv shape:", df2.shape)
display(df1.head())
display(df2.head())


homework_2.1.csv shape: (100, 5)
homework_2.2.csv shape: (10000, 4)


,Unnamed: 0,time,G1,G2,G3
0,0,0,0.882026,1.441575,0.065409
1,1,1,0.210079,-0.163880,0.140310
2,2,2,0.509369,-0.115242,0.819830
3,3,3,1.150447,1.014698,0.607632
4,4,4,0.973779,-0.046562,0.610066


,Unnamed: 0,X,Y,Z
0,0,0,1.182435,-0.725820
1,1,0,2.714474,0.563476
2,2,0,0.077612,-0.435632
3,3,0,-0.154449,-0.104553
4,4,0,22.298992,-2.321273


In [3]:
# --- Q1 & Q2: Fixed effects with one common linear coefficient on 'time' ---
# Reshape wide (G1, G2, G3) to long to fit one model with group fixed effects
df_long = pd.melt(
    df1,
    id_vars=['time'],
    value_vars=['G1', 'G2', 'G3'],
    var_name='group',
    value_name='value'
)

# Use G1 as the reference group so Intercept == Group 1's fixed effect
formula = 'value ~ time + C(group, Treatment(reference="G1"))'
model = smf.ols(formula, data=df_long).fit()

print("--- Regression Summary for Q1 & Q2 ---")
print(model.summary())

# Extract target coefficients
intercept_g1 = model.params['Intercept']     # Group 1 fixed effect
common_beta  = model.params['time']          # Common linear coefficient

print("\n--- Extracted Coefficients ---")
print(f"Group 1 fixed effect (Intercept) = {intercept_g1:.8f}")
print(f"Common linear coefficient on time = {common_beta:.8f}")

# Nearest option helper (multiple-choice mapping)
def pick_nearest(x, options, labels=('A','B','C','D')):
    idx = int(np.argmin(np.abs(np.array(options) - x)))
    return labels[idx], options[idx]

q1_options = [0.00485, 0.00850, 0.01023, 0.1823]
q2_options = [0.01003, 0.009017, 0.01120, 0.2808]

q1_pick_label, q1_pick_value = pick_nearest(intercept_g1, q1_options)
q2_pick_label, q2_pick_value = pick_nearest(common_beta, q2_options)

print("\n--- Nearest Multiple-Choice Picks ---")
print(f"Q1: {q1_pick_label} (closest to {q1_pick_value})")
print(f"Q2: {q2_pick_label} (closest to {q2_pick_value})")


--- Regression Summary for Q1 & Q2 ---
                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     44.55
Date:                Sat, 06 Sep 2025   Prob (F-statistic):           8.72e-24
Time:                        21:14:01   Log-Likelihood:                -216.89
No. Observations:                 300   AIC:                             441.8
Df Residuals:                     296   BIC:                             456.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

In [12]:
# --- Q3–Q5: Bootstrap tasks on homework_2.2.csv ---
# Columns: X (treatment indicator), Y (outcome), Z (confounder)

# Q3: Simple difference in means (not recommended if confounders exist)
mean_treated = df2.loc[df2['X'] == 1, 'Y'].mean()
mean_control = df2.loc[df2['X'] == 0, 'Y'].mean()
effect_diff_means = mean_treated - mean_control
print(f"Q3 — Diff-in-means effect = {effect_diff_means:.8f}")

# Helper to compute diff-in-means on a sample DataFrame
def diff_in_means_effect(sample_df):
    # Guard against rare cases with no treated or no control in a bootstrap draw
    if (sample_df['X'] == 1).sum() == 0 or (sample_df['X'] == 0).sum() == 0:
        return np.nan
    return sample_df.loc[sample_df['X'] == 1, 'Y'].mean() - sample_df.loc[sample_df['X'] == 0, 'Y'].mean()

# Q4: Bootstrap variance of the (non-recommended) diff-in-means estimator
n = len(df2)
n_boot = 100000  # Increase for higher stability (e.g., 10000)
boot_effects = np.empty(n_boot)

for b in range(n_boot):
    idx = rng.integers(0, n, size=n)  # sample rows with replacement
    s = df2.iloc[idx]
    val = diff_in_means_effect(s)
    # If a pathological sample occurs, resample once
    if np.isnan(val):
        idx = rng.integers(0, n, size=n)
        s = df2.iloc[idx]
        val = diff_in_means_effect(s)
    boot_effects[b] = val

# Remove any remaining NaNs just in case
boot_effects = boot_effects[~np.isnan(boot_effects)]
var_boot = np.var(boot_effects, ddof=1)
print(f"Q4 — Bootstrap variance of diff-in-means = {var_boot:.8f}")

# Q5: Bootstrap the regression coefficient from Y ~ 1 + X and compute skewness
def reg_coef_X(sample_df):
    X = sm.add_constant(sample_df['X'].to_numpy())
    y = sample_df['Y'].to_numpy()
    return sm.OLS(y, X).fit().params[1]  # coefficient on X

boot_betas = np.empty(n_boot)
for b in range(n_boot):
    idx = rng.integers(0, n, size=n)
    s = df2.iloc[idx]
    boot_betas[b] = reg_coef_X(s)

skew_boot = skew(boot_betas, bias=False)  # unbiased estimator
print(f"Q5 — Skewness of bootstrap distribution (beta_X) = {skew_boot:.8f}")

# Multiple-choice mapping for Q3–Q5
q3_options = [12.831, 5.149, 2.921, 8.031]
q4_options = [0.002388, 0.03274, 0.001822, 0.1280]
q5_options = [0.2315, 0.04850, 0.3223, 0.09812]

q3_pick_label, q3_pick_value = pick_nearest(effect_diff_means, q3_options)
q4_pick_label, q4_pick_value = pick_nearest(var_boot, q4_options)
q5_pick_label, q5_pick_value = pick_nearest(skew_boot, q5_options)

print("\n--- Nearest Multiple-Choice Picks ---")
print(f"Q3: {q3_pick_label} (closest to {q3_pick_value})")
print(f"Q4: {q4_pick_label} (closest to {q4_pick_value})")
print(f"Q5: {q5_pick_label} (closest to {q5_pick_value})")


Q3 — Diff-in-means effect = 2.92071726
Q4 — Bootstrap variance of diff-in-means = 0.03168891
Q5 — Skewness of bootstrap distribution (beta_X) = 0.04390341

--- Nearest Multiple-Choice Picks ---
Q3: C (closest to 2.921)
Q4: B (closest to 0.03274)
Q5: B (closest to 0.0485)
